In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json
from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [2]:
def fun(x):
    if x < -50:
        return 'Strong Sell'
    elif x > -50  and x < -10:
        return 'Sell'
    elif x > -10 and x < 10:
        return 'Neutral'
    elif x > 10 and x < 50:
        return 'Buy'
    else:
        return 'Strong Buy'
    
def fun1(x,v1,v2,v3,v4):
    if x < v1:
        return 'Strong Sell'
    elif x > v1 and x < v2:
        return 'Sell'
    elif x > v2 and x < v3:
        return 'Neutral'
    elif x > v3 and x < v4:
        return 'Buy'
    else:
        return 'Strong Buy'
    
    
def acc(x):
    if x.name == 'Strong Sell' or x.name == 'Sell':
        return (x['Strong Sell'] + x['Sell']) / x['Total']
    elif x.name == 'Neutral':
        return (x['Neutral']) / x['Total']
    else:
        return (x['Strong Buy'] + x['Buy']) / x['Total']

In [3]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
candlestick = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    
    query = '''SELECT * FROM ianbalina.ico_technical_analyses;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.technical_analysis_scoring;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.technical_analysis_weights;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    query = '''SELECT * FROM ianbalina.ico_candlestick_patterns;'''
    data = pd.read_sql_query(query, conn)
    candlestick = pd.concat([candlestick,data])
    
    conn.close() 

I1004 07:28:55.354453   584 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I1004 07:28:57.536217   584 transport.py:1819] Authentication (password) successful!


### Weekly

In [4]:
d3['indicator'].iloc[11] = 'ichimoku_lagging_span'
candlestick['signal'] = candlestick['signal'].replace(['bullish','bearish','bearish_bullish'],[25,-25,0])
d1 = d1[d1['date'] == datetime.date(2019,8,19)]

In [24]:
t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,8,19)][t1['date'] <= datetime.date(2019,8,27)]
    t1 = t1.drop_duplicates(subset=['date'])
    #t1 = t1.sort_values('date')
    if len(t1) > 1:
        x1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        x2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,x1,x2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

In [25]:
def get_result(x = [4,2,1,11,1,3,7,2,1,10,7,21,25,6]):
    d3['weight'] = x
    d4 = pd.merge(d1,d3[['indicator','weight']], on=['indicator'] ,how='left')
    d4['signal'] = d4['signal'].replace(['neutral','downward','upward','buy','sell'],[0,-25,25,75,-75])

    
    temp = pd.DataFrame()
    for coin in df['ico_symbol'].unique().tolist():
        num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
        d5 = d4[d4['ico_id'].isin(num)]
        d5 = d5[['indicator','weight','signal']]

        candlestick['weight'] = 1
        t1 = candlestick[candlestick['ico_id'].isin(num)]
        t1 = t1[['pattern','weight','signal']]
        t1.columns = d5.columns

        d5 = pd.concat([d5,t1])

        d5 = d5.drop_duplicates('indicator')
        d5['weight'] = d5['weight'].fillna(1)
        d5 = d5.fillna(0)
        temp = pd.concat([temp,pd.DataFrame([coin,sum(d5['weight'] * d5['signal'].astype('float64')) / d5['weight'].sum()])], axis=1)
        
        
    temp = temp.T
    temp.columns = ['Coins','Technical Score']
    temp.dropna(inplace=True)

    
    final = pd.merge(temp,t2)
    
    final['TA result'] = final['Technical Score'].apply(fun)
    final['Actual result'] = final['% Change'].apply(fun1,args=[-15,-5,5,15])
    final = final[final['Technical Score'] != 0]
    data = pd.crosstab(final['TA result'], final['Actual result'])

    data = data.reindex(['Strong Sell','Sell','Neutral','Buy','Strong Buy'])
    data = data.reindex(columns=['Strong Sell','Sell','Neutral','Buy','Strong Buy'])

    x1 = data.sum(axis=1)
    data['Total'] = x1
    x2 = data.T.apply(acc) * 100
    data['Result'] = (x1*x2)/100
    data['% Accurate'] = x2
    data.fillna(0,inplace=True)
    
    data.index = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15+)%']
    data.columns = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15+)%', 'Total', 'Result', '% Accurate']

    return data

In [9]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,2,0,5,0,0,7,2.000000,28.571429
(-15 - -5)%,0,1,14,0,0,15,1.000000,6.666667
(-5 - 5)%,7,13,76,6,5,107,76.000000,71.028037
(5 - 15)%,1,0,53,7,1,62,8.000000,12.903226
(15)%,2,4,65,6,2,79,8.000000,10.126582


In [10]:
x['Result'].sum() / x['Total'].sum()

0.3518518518518518

In [26]:
d3['weight'] = [11, 3, 4, 15, 5, 3, 5, 1, 23, 15, 14, 19, 2, 19]
d3

,id,indicator,weight,created_at,updated_at
0,1,adx,11,None,None
1,2,ao,3,None,None
2,3,cci,4,None,None
3,4,macd,15,None,None
4,5,fi,5,None,None
5,6,rsi,3,None,None
6,7,kd,5,None,None
7,8,wr,1,None,None
8,9,roc,23,None,None
9,10,trix,15,None,None


In [27]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,1.000000,4.000000,0.000000,0.000000,5.000000,1.000000,20.000000
(-5 - 5)%,12.000000,16.000000,194.000000,19.000000,6.000000,247.000000,194.000000,78.542510
(5 - 15)%,0.000000,1.000000,14.000000,0.000000,1.000000,16.000000,1.000000,6.250000
(15+)%,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,50.000000


In [28]:
x['Result'].sum() / x['Total'].sum()

0.7296296296296295

In [32]:
d3['weight'] = [3, 13, 13, 22, 8, 5, 21, 3, 0, 22, 4, 7, 13, 21]
d3

,id,indicator,weight,created_at,updated_at
0,1,adx,3,None,None
1,2,ao,13,None,None
2,3,cci,13,None,None
3,4,macd,22,None,None
4,5,fi,8,None,None
5,6,rsi,5,None,None
6,7,kd,21,None,None
7,8,wr,3,None,None
8,9,roc,0,None,None
9,10,trix,22,None,None


In [33]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,3.000000,1.000000,11.000000,0.000000,0.000000,15.000000,4.000000,26.666667
(-5 - 5)%,7.000000,13.000000,90.000000,6.000000,6.000000,122.000000,90.000000,73.770492
(5 - 15)%,2.000000,4.000000,109.000000,13.000000,1.000000,129.000000,14.000000,10.852713
(15+)%,0.000000,0.000000,2.000000,0.000000,1.000000,3.000000,1.000000,33.333333


In [34]:
x['Result'].sum() / x['Total'].sum()

0.4052044609665427

### Rolling Basis

In [4]:
d3['indicator'].iloc[11] = 'ichimoku_lagging_span'
candlestick['signal'] = candlestick['signal'].replace(['bullish','bearish','bearish_bullish'],[25,-25,0])

In [5]:
def get_result(d1,x = [4,2,1,11,1,3,7,2,1,10,7,21,25,6]):
    d3['weight'] = x
    d4 = pd.merge(d1,d3[['indicator','weight']], on=['indicator'] ,how='left')
    d4['signal'] = d4['signal'].replace(['neutral','downward','upward','buy','sell','bullish','bearish'],[0,-25,25,75,-75,25,-25])
    
    temp = pd.DataFrame()
    for coin in df['ico_symbol'].unique().tolist():
        num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
        d5 = d4[d4['ico_id'].isin(num)]
        d5 = d5[['indicator','weight','signal']]

        candlestick['weight'] = 1
        t1 = candlestick[candlestick['ico_id'].isin(num)]
        t1 = t1[['pattern','weight','signal']]
        t1.columns = d5.columns

        d5 = pd.concat([d5,t1])

        d5 = d5.drop_duplicates('indicator')
        d5['weight'] = d5['weight'].fillna(1)
        d5 = d5.fillna(0)
        temp = pd.concat([temp,pd.DataFrame([coin,sum(d5['weight'] * d5['signal'].astype('float64')) / d5['weight'].sum()])], axis=1)
        
        
    temp = temp.T
    temp.columns = ['Coins','Technical Score']
    temp.dropna(inplace=True)

    
    final = pd.merge(temp,t2)
    
    final['TA result'] = final['Technical Score'].apply(fun)
    final['Actual result'] = final['% Change'].apply(fun1,args=[-15,-5,5,15])
    final = final[final['Technical Score'] != 0]
    data = pd.crosstab(final['TA result'], final['Actual result'])

    data = data.reindex(['Strong Sell','Sell','Neutral','Buy','Strong Buy'])
    data = data.reindex(columns=['Strong Sell','Sell','Neutral','Buy','Strong Buy'])

    x1 = data.sum(axis=1)
    data['Total'] = x1
    x2 = data.T.apply(acc) * 100
    data['Result'] = (x1*x2)/100
    data['% Accurate'] = x2
    data.fillna(0,inplace=True)
    
    data.index = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15+)%']
    data.columns = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15+)%', 'Total', 'Result', '% Accurate']

    return data

In [37]:
temp = d1[d1['date'] == datetime.date(2019,9,16)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,16)][t1['date'] <= datetime.date(2019,9,23)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x7 = get_result(temp,[20, 1, 18, 9, 0, 5, 25, 8, 4, 8, 10, 20, 23, 8])
print(x7['Result'].sum() / x7['Total'].sum())

0.43902439024390233


In [7]:
temp = d1[d1['date'] == datetime.date(2019,9,17)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,17)][t1['date'] <= datetime.date(2019,9,24)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x6 = get_result(temp,[3, 16, 3, 21, 12, 0, 6, 1, 1, 0, 24, 11, 5, 4])
print(x6['Result'].sum() / x6['Total'].sum())

0.3987915407854985


In [8]:
temp = d1[d1['date'] == datetime.date(2019,9,18)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,18)][t1['date'] <= datetime.date(2019,9,25)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x5 = get_result(temp,[8, 25, 1, 4, 5, 1, 7, 4, 7, 2, 20, 1, 4, 3])
print(x5['Result'].sum() / x5['Total'].sum())

0.3806646525679758


In [9]:
temp = d1[d1['date'] == datetime.date(2019,9,20)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,20)][t1['date'] <= datetime.date(2019,9,27)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x4 = get_result(temp,[7, 13, 24, 0, 8, 2, 17, 21, 5, 0, 16, 12, 5, 2])
print(x4['Result'].sum() / x4['Total'].sum())

0.24758842443729903


In [10]:
temp = d1[d1['date'] == datetime.date(2019,9,21)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,21)][t1['date'] <= datetime.date(2019,9,28)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x3 = get_result(temp,[2, 18, 17, 25, 16, 7, 25, 7, 16, 5, 7, 1, 16, 1])
print(x3['Result'].sum() / x3['Total'].sum())

0.2951807228915663


In [11]:
temp = d1[d1['date'] == datetime.date(2019,9,22)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,22)][t1['date'] <= datetime.date(2019,9,29)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x2 = get_result(temp, [3, 20, 0, 9, 13, 2, 18, 3, 19, 2, 18, 2, 4, 11])
print(x2['Result'].sum() / x2['Total'].sum())

0.29003021148036257


In [12]:
temp = d1[d1['date'] == datetime.date(2019,9,23)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,23)][t1['date'] <= datetime.date(2019,9,30)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x1 = get_result(temp,[19, 19, 16, 5, 23, 5, 22, 17, 24, 3, 8, 1, 0, 7])
print(x1['Result'].sum() / x1['Total'].sum())

0.3373860182370821


In [38]:
data = pd.DataFrame()
data['date'] = ['(19-26)-Sept','(20-27)-Sept','(21-28)-Sept','(22-29)-Sept','(23-30)-Sept','(24 Sept) - (1 Oct)',
               '(25 Sept) - (2 Oct)']
data['Accuracy'] = [x7['Result'].sum() / x7['Total'].sum(),x6['Result'].sum() / x6['Total'].sum(),
                    x5['Result'].sum() / x5['Total'].sum(),x4['Result'].sum() / x4['Total'].sum(),
                   x3['Result'].sum() / x3['Total'].sum(),x2['Result'].sum() / x2['Total'].sum(),
                   x1['Result'].sum() / x1['Total'].sum()]

In [39]:
data

,date,Accuracy
0,(19-26)-Sept,0.439024
1,(20-27)-Sept,0.398792
2,(21-28)-Sept,0.380665
3,(22-29)-Sept,0.247588
4,(23-30)-Sept,0.295181
5,(24 Sept) - (1 Oct),0.290030
6,(25 Sept) - (2 Oct),0.337386


In [15]:
x3

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,58.000000,17.000000,7.000000,0.000000,1.000000,83.000000,75.000000,90.361446
(-5 - 5)%,135.000000,39.000000,22.000000,7.000000,5.000000,208.000000,22.000000,10.576923
(5 - 15)%,33.000000,6.000000,1.000000,0.000000,1.000000,41.000000,1.000000,2.439024
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
x4

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,48.000000,4.000000,3.000000,0.000000,1.000000,56.000000,52.000000,92.857143
(-5 - 5)%,171.000000,33.000000,25.000000,4.000000,4.000000,237.000000,25.000000,10.548523
(5 - 15)%,14.000000,3.000000,1.000000,0.000000,0.000000,18.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
x1

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,61.000000,17.000000,4.000000,0.000000,3.000000,85.000000,78.000000,91.764706
(-5 - 5)%,123.000000,56.000000,33.000000,6.000000,10.000000,228.000000,33.000000,14.473684
(5 - 15)%,7.000000,2.000000,5.000000,0.000000,0.000000,14.000000,0.000000,0.000000
(15+)%,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000


In [18]:
x2

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,66.000000,8.000000,4.000000,0.000000,1.000000,79.000000,74.000000,93.670886
(-5 - 5)%,165.000000,36.000000,22.000000,6.000000,6.000000,235.000000,22.000000,9.361702
(5 - 15)%,13.000000,2.000000,2.000000,0.000000,0.000000,17.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
x5

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,91.000000,19.000000,0.000000,2.000000,3.000000,115.000000,110.000000,95.652174
(-5 - 5)%,140.000000,31.000000,16.000000,6.000000,4.000000,197.000000,16.000000,8.121827
(5 - 15)%,13.000000,1.000000,4.000000,0.000000,0.000000,18.000000,0.000000,0.000000
(15+)%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [20]:
x6

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,75.000000,18.000000,9.000000,3.000000,4.000000,109.000000,93.000000,85.321101
(-5 - 5)%,113.000000,42.000000,39.000000,5.000000,6.000000,205.000000,39.000000,19.024390
(5 - 15)%,11.000000,2.000000,4.000000,0.000000,0.000000,17.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
x7

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,1.000000,10.000000,16.000000,7.000000,6.000000,40.000000,11.000000,27.500000
(-5 - 5)%,21.000000,53.000000,129.000000,45.000000,25.000000,273.000000,129.000000,47.252747
(5 - 15)%,0.000000,3.000000,8.000000,4.000000,0.000000,15.000000,4.000000,26.666667
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
data = pd.DataFrame()
data['date'] = ['(19-26)-Sept','(20-27)-Sept','(21-28)-Sept','(22-29)-Sept','(23-30)-Sept','(24 Sept) - (1 Oct)',
               '(25 Sept) - (2 Oct)']
data['Accuracy'] = [(x7['Result'].sum() - x7['Result'].iloc[2]) / (x7['Total'].sum() - x7['Total'].iloc[2]),
                    (x6['Result'].sum() - x6['Result'].iloc[2]) / (x6['Total'].sum() - x6['Total'].iloc[2]),
                    (x5['Result'].sum() - x5['Result'].iloc[2]) / (x5['Total'].sum() - x5['Total'].iloc[2]),
                   (x4['Result'].sum() - x4['Result'].iloc[2]) / (x4['Total'].sum() - x4['Total'].iloc[2]),
                   (x3['Result'].sum() - x3['Result'].iloc[2]) / (x3['Total'].sum() - x3['Total'].iloc[2]),
                   (x2['Result'].sum() - x2['Result'].iloc[2]) / (x2['Total'].sum() - x2['Total'].iloc[2]),
                   (x1['Result'].sum() - x1['Result'].iloc[2]) / (x1['Total'].sum() - x1['Total'].iloc[2])]

In [32]:
(x6['Result'].sum() - x6['Result'].iloc[2]) / (x6['Total'].sum() - x6['Total'].iloc[2])

0.7380952380952381

In [33]:
data

,date,Accuracy
0,(19-26)-Sept,0.272727
1,(20-27)-Sept,0.738095
2,(21-28)-Sept,0.820896
3,(22-29)-Sept,0.702703
4,(23-30)-Sept,0.612903
5,(24 Sept) - (1 Oct),0.770833
6,(25 Sept) - (2 Oct),0.772277


### Test

In [51]:
temp = d1[d1['date'] == datetime.date(2019,9,26)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,26)][t1['date'] <= datetime.date(2019,10,3)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x1 = get_result(temp,[0, 1, 20, 4, 18, 7, 1, 7, 3, 19, 1, 22, 23, 0])
print((x1['Result'].sum() - x1['Result'].iloc[2]) / (x1['Total'].sum() - x1['Total'].iloc[2]))

0.3597560975609756


In [49]:
temp = d1[d1['date'] == datetime.date(2019,9,25)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,25)][t1['date'] <= datetime.date(2019,10,2)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x2 = get_result(temp,[3, 2, 8, 1, 11, 16, 0, 4, 8, 5, 0, 17, 10, 8])
print((x2['Result'].sum() - x2['Result'].iloc[2]) / (x2['Total'].sum() - x2['Total'].iloc[2]))

0.3783783783783784


In [45]:
temp = d1[d1['date'] == datetime.date(2019,9,24)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,24)][t1['date'] <= datetime.date(2019,10,1)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x3 = get_result(temp,[25, 5, 5, 0, 15, 20, 9, 10, 18, 16, 1, 8, 24, 12])
print((x3['Result'].sum() - x3['Result'].iloc[2]) / (x3['Total'].sum() - x3['Total'].iloc[2]))

0.5789473684210527


In [23]:
temp = d1[d1['date'] == datetime.date(2019,9,23)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,23)][t1['date'] <= datetime.date(2019,9,30)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x4 = get_result(temp,[13, 13, 21, 21, 20, 9, 9, 17, 10, 1, 16, 18, 0, 8])
print((x4['Result'].sum() - x4['Result'].iloc[2]) / (x4['Total'].sum() - x4['Total'].iloc[2]))

0.7804878048780489


In [28]:
temp = d1[d1['date'] == datetime.date(2019,9,22)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,22)][t1['date'] <= datetime.date(2019,9,29)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x5 = get_result(temp,[13, 25, 2, 4, 7, 8, 18, 13, 6, 8, 8, 16, 3, 22])
print((x5['Result'].sum() - x5['Result'].iloc[2]) / (x5['Total'].sum() - x5['Total'].iloc[2]))

0.8846153846153846


In [30]:
temp = d1[d1['date'] == datetime.date(2019,9,21)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,21)][t1['date'] <= datetime.date(2019,9,28)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x6 = get_result(temp,[13, 25, 2, 4, 7, 8, 18, 13, 6, 8, 8, 16, 3, 22])
print((x6['Result'].sum() - x6['Result'].iloc[2]) / (x6['Total'].sum() - x6['Total'].iloc[2]))

0.818181818181818


In [31]:
temp = d1[d1['date'] == datetime.date(2019,9,20)]

t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,9,20)][t1['date'] <= datetime.date(2019,9,27)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        y1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        y2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,y1,y2])], axis=1)


t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

x7 = get_result(temp,[13, 25, 2, 4, 7, 8, 18, 13, 6, 8, 8, 16, 3, 22])
print((x7['Result'].sum() - x7['Result'].iloc[2]) / (x7['Total'].sum() - x7['Total'].iloc[2]))

0.8372093023255812


In [52]:
data = pd.DataFrame()
data['date'] = ['(21-28)-Sept','(22-29)-Sept','(23-30)-Sept','(24 Sept) - (1 Oct)','(25 Sept) - (2 Oct)','(26 Sept) - (3 Oct)',
               '(27 Sept) - (4 Oct)']
data['Accuracy'] = [(x7['Result'].sum() - x7['Result'].iloc[2]) / (x7['Total'].sum() - x7['Total'].iloc[2]),
                    (x6['Result'].sum() - x6['Result'].iloc[2]) / (x6['Total'].sum() - x6['Total'].iloc[2]),
                    (x5['Result'].sum() - x5['Result'].iloc[2]) / (x5['Total'].sum() - x5['Total'].iloc[2]),
                   (x4['Result'].sum() - x4['Result'].iloc[2]) / (x4['Total'].sum() - x4['Total'].iloc[2]),
                   (x3['Result'].sum() - x3['Result'].iloc[2]) / (x3['Total'].sum() - x3['Total'].iloc[2]),
                   (x2['Result'].sum() - x2['Result'].iloc[2]) / (x2['Total'].sum() - x2['Total'].iloc[2]),
                   (x1['Result'].sum() - x1['Result'].iloc[2]) / (x1['Total'].sum() - x1['Total'].iloc[2])]

In [53]:
data

,date,Accuracy
0,(21-28)-Sept,0.837209
1,(22-29)-Sept,0.818182
2,(23-30)-Sept,0.884615
3,(24 Sept) - (1 Oct),0.780488
4,(25 Sept) - (2 Oct),0.578947
5,(26 Sept) - (3 Oct),0.378378
6,(27 Sept) - (4 Oct),0.359756


In [35]:
x7.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,33.000000,3.000000,1.000000,0.000000,1.000000,38.000000,36.000000,94.736842
(5 - 15)%,4.000000,0.000000,1.000000,0.000000,0.000000,5.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
x6.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,28.000000,7.000000,0.000000,1.000000,0.000000,36.000000,35.000000,97.222222
(5 - 15)%,3.000000,3.000000,1.000000,0.000000,1.000000,8.000000,1.000000,12.500000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
x5.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,18.000000,5.000000,1.000000,0.000000,0.000000,24.000000,23.000000,95.833333
(5 - 15)%,1.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
x4.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,25.000000,7.000000,2.000000,1.000000,1.000000,36.000000,32.000000,88.888889
(5 - 15)%,1.000000,0.000000,4.000000,0.000000,0.000000,5.000000,0.000000,0.000000
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [63]:
x3.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,1.000000,2.000000,0.000000,2.000000,1.000000,6.000000,3.000000,50.000000
(5 - 15)%,2.000000,1.000000,2.000000,4.000000,4.000000,13.000000,8.000000,61.538462
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [64]:
x2.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,0.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000
(5 - 15)%,3.000000,6.000000,10.000000,8.000000,6.000000,33.000000,14.000000,42.424242
(15+)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [65]:
x1.drop('(-5 - 5)%')

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15+)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,1.000000,3.000000,2.000000,1.000000,7.000000,1.000000,14.285714
(5 - 15)%,10.000000,16.000000,72.000000,44.000000,14.000000,156.000000,58.000000,37.179487
(15+)%,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
